# Dataloader Setup

In [3]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

Global seed set to 1234


1234

# Model

In [4]:
from xnornet_gap import Model
binary_structure = [{'in_channels':3, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'no'},
             {'in_channels':128, 'out_channels':128, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'no'},
             {'in_channels':128, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'no'},
             {'in_channels':256, 'out_channels':256, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'no'},
             {'in_channels':256, 'out_channels':512, 'stride':1, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'no'},
             {'in_channels':512, 'out_channels':512, 'stride':2, 'kernel_size':3, 'padding':1, 'conv':'scaled_sign', 'act_fn':'sign', 'dropout':0,'avgpool':'pool'}, # 6 conv
             {'in_channels':512, 'out_channels':10, 'stride':1, 'kernel_size':1, 'padding':0, 'conv':'scaled_sign', 'act_fn':'none', 'dropout':0,'avgpool':'no'}]


model = Model(structure=binary_structure, 
              adam_init_lr=1e-2, 
              lr_patience=50,
              limit_conv_weight=True,
              limit_bn_weight=False)
print(model)
print(model.hparams)

Model(
  (blocks): ModuleList(
    (0): BasicBlock(
      (GeneralConv2d): GeneralConv2d(3, 128, kernel_size=3, stride=1, padding=1, conv=scaled_sign)
      (BatchNorm2d): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (Shift): Shift(out_channels=128)
      (Clamp): Clamp
      (BinaryActivation): BinaryActivation()
    )
    (1): BasicBlock(
      (GeneralConv2d): GeneralConv2d(128, 128, kernel_size=3, stride=1, padding=1, conv=scaled_sign)
      (MaxPool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (BatchNorm2d): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (Shift): Shift(out_channels=128)
      (Clamp): Clamp
      (BinaryActivation): BinaryActivation()
    )
    (2): BasicBlock(
      (GeneralConv2d): GeneralConv2d(128, 256, kernel_size=3, stride=1, padding=1, conv=scaled_sign)
      (BatchNorm2d): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_

# Initialize Parameters from Real Ones

In [ ]:
#model = Model.load_from_checkpoint('lightning_logs/no_affine_bias/version_0/checkpoints/epoch=422-val_loss=0.6115-val_acc=0.8388.ckpt')
#model.load_state_dict(real_model.state_dict(), strict=False)

# Training & Testing

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='no_affine_weight + no_shiftlayer', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(model, datamodule=cifar10_dm)

In [ ]:
trainer.test(model, datamodule=cifar10_dm)
print(checkpoint_callback.best_model_score)
print(checkpoint_callback.best_model_path)

In [ ]:
quit()